# Calculate ILAMB Data
Creates annual, regridded data from raw ILAMB data. Uses a config file (e.g. ILAMB_obs.cfg) to get information about where ILAMB data is located and what the files, models, and variable names are.

In [70]:
import os
import xarray as xr

import fates_calibration_library.utils
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.analysis_functions as analysis

In [125]:
import xesmf as xe
import numpy as np
import importlib
importlib.reload(fates_calibration_library.analysis_functions)
importlib.reload(fates_calibration_library.ilamb_functions)

<module 'fates_calibration_library.ilamb_functions' from '/glade/work/afoster/FATES_calibration/scripts/fates_calibration_library/ilamb_functions.py'>

In [133]:
# config file for ILAMB data
ILAMB_cfg = '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs.cfg'

# land area file
land_area_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")

In [134]:
# config dictionary for running functions
config_dict = {
    'top_dir': '/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA',  # location of raw ILAMB data
    'out_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs',  # where to put compiled ILAMB data
    'start_date': '2000-01-01',  # start date to filter to
    'end_date': '2014-12-31',  # end date to filter to
    'user': 'afoster@ucar.edu',  # user
    'clobber': True,  # overwrite files?
}

In [135]:
# read in the ILAMB data dictionary
ILAMB_dict = fates_calibration_library.utils.config_to_dict(ILAMB_cfg)

In [136]:
ILAMB_dict

{'FLUXCOM_GPP': {'sub_dir': 'gpp',
  'model': 'FLUXCOM',
  'filename': 'gpp.nc',
  'in_var': 'gpp',
  'out_var': 'gpp',
  'conversion_factor': '0.001',
  'area_conversion_factor': '1e-6',
  'max_val': '500',
  'min_val': 'None',
  'units': 'kgC m-2 yr-1',
  'longname': 'gross primary production'},
 'WECANN_GPP': {'sub_dir': 'gpp',
  'model': 'WECANN',
  'filename': 'gpp.nc',
  'in_var': 'gpp',
  'out_var': 'gpp',
  'conversion_factor': '0.001',
  'area_conversion_factor': '1e-6',
  'max_val': '500',
  'min_val': 'None',
  'units': 'kgC m-2 yr-1',
  'longname': 'gross primary production'},
 'GBAF_GPP': {'sub_dir': 'gpp',
  'model': 'GBAF',
  'filename': 'gpp_0.5x0.5.nc',
  'in_var': 'gpp',
  'out_var': 'gpp',
  'conversion_factor': '86400.0',
  'area_conversion_factor': '1e-6',
  'max_val': 'None',
  'min_val': 'None',
  'units': 'kgC m-2 yr-1',
  'longname': 'gross primary production'},
 'FLUXCOM_NEE': {'sub_dir': 'nee',
  'model': 'FLUXCOM',
  'filename': 'nee.nc',
  'in_var': 'nee',


In [129]:
# create a target grid to regrid to
target_grid = analysis.create_target_grid(land_area_ds, 'FSR')

In [137]:
attributes = ILAMB_dict['FLUXCOM_GPP']

In [131]:
model = attributes["model"]
out_var = attributes["out_var"]
file_name = f"{model}_{out_var.upper()}.nc"
out_file = os.path.join(config_dict["out_dir"], file_name)

ilamb_dat, original_file = ilamb.read_ilamb_data(
    config_dict["top_dir"], ILAMB_dict, attributes, config_dict
)

metadata = {
    "units": attributes["units"],
    "longname": attributes["longname"],
    "original_file": original_file,
    "user": config_dict["user"],
}
# convert to annual dataset
annual_ds = ilamb.get_annual_ds(
    ilamb_dat,
    attributes["in_var"],
    out_var,
    ilamb.get_conversion_factor(attributes["conversion_factor"]),
    metadata,
)

# monthly mean
monthly_mean = ilamb.calculate_monthly_mean(
    ilamb_dat[attributes["in_var"]],
    ilamb.get_conversion_factor(attributes["conversion_factor"]),
)

month_of_max = analysis.get_monthly_max(monthly_mean).to_dataset(name = f"{attributes['out_var']}_month_of_max")
regridded_monthly = ilamb.regrid_ilamb_ds(month_of_max, target_grid, f"{attributes['out_var']}_month_of_max", method='nearest_s2d')

In [138]:
# get climatology
climatology_ds = ilamb.get_ilamb_climatology(monthly_mean, 
                                       area_cf=ilamb.get_conversion_factor(attributes["area_conversion_factor"]))

In [139]:
regridded_annual = ilamb.regrid_ilamb_ds(annual_ds, target_grid, attributes["out_var"])

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


In [142]:
ds_out = xr.merge([regridded_monthly, regridded_annual, climatology_ds])

In [143]:
ds_out

<xarray.Dataset>
Dimensions:           (lat: 96, lon: 144, year: 14, month: 12)
Coordinates:
  * lon               (lon) float32 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lat               (lat) float32 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
    landmask          (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    landfrac          (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
  * year              (year) int64 2000 2001 2002 2003 ... 2010 2011 2012 2013
  * month             (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    gpp_month_of_max  (lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    gpp               (year, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    gpp_cycle         (month) float64 7.574 7.165 8.257 ... 8.204 7.223 7.308
    gpp_anomaly       (month) float64 -1.98 -2.389 -1.298 ... -2.331 -2.247

In [ ]:
# loop through dictionary and create the regridded data
# script will skip files that already exist unless clobber=True
ilamb.get_all_ilamb_data(config_dict, ILAMB_dict, target_grid)

In [ ]:
# compile data into one dataset - also averages by year
compiled_ds = ilamb.compile_ilamb_datasets(config_dict['out_dir'], ILAMB_dict,
                                           land_area_ds)
compiled_ds.to_netcdf('/glade/work/afoster/FATES_calibration/observations/all_ILAMB_obs.nc',
                      mode="w")